In [0]:
CREATE OR REPLACE VIEW fact_questionnaire_long AS
WITH
-- -------- Non-AI (Q1-12) --------
n_base AS (SELECT * FROM n_q1_q12_clean),

n_scores AS (
  SELECT
    drive_file_id, file_name, file_link, school_id, ai_used, page_num, respondent_key,
    cast(regexp_extract(q_col, 'q(\\d+)', 1) as int) AS question_num,
    score
  FROM n_base
  UNPIVOT (score FOR q_col IN (q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12))
),

n_long AS (
  SELECT
    s.drive_file_id, s.file_name, s.file_link, s.school_id, s.ai_used, s.page_num, s.respondent_key,
    s.question_num,
    concat('q', s.question_num) AS question_code,
    s.score,
    cast(NULL as string) AS text_answer
  FROM n_scores s
),

-- -------- AI (Q1-12) --------
y12_base AS (SELECT * FROM y_q1_q12_clean),

y12_scores AS (
  SELECT
    drive_file_id, file_name, file_link, school_id, ai_used, page_num, respondent_key,
    cast(regexp_extract(q_col, 'q(\\d+)', 1) as int) AS question_num,
    score
  FROM y12_base
  UNPIVOT (score FOR q_col IN (q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12))
),

y12_long AS (
  SELECT
    s.drive_file_id, s.file_name, s.file_link, s.school_id, s.ai_used, s.page_num, s.respondent_key,
    s.question_num,
    concat('q', s.question_num) AS question_code,
    s.score,
    cast(NULL as string) AS text_answer
  FROM y12_scores s
),

-- -------- AI (Q13-17 numeric) --------
y18_base AS (SELECT * FROM y_q13_q18_clean),

y13_17_scores AS (
  SELECT
    drive_file_id, file_name, file_link, school_id, ai_used, page_num, respondent_key,
    cast(regexp_extract(q_col, 'q(\\d+)', 1) as int) AS question_num,
    score
  FROM y18_base
  UNPIVOT (score FOR q_col IN (q13,q14,q15,q16,q17))
),

y13_17_long AS (
  SELECT
    s.drive_file_id, s.file_name, s.file_link, s.school_id, s.ai_used, s.page_num, s.respondent_key,
    s.question_num,
    concat('q', s.question_num) AS question_code,
    s.score,
    cast(NULL as string) AS text_answer
  FROM y13_17_scores s
),

-- -------- AI (Q18 free-text) --------
q18_long AS (
  SELECT
    drive_file_id, file_name, file_link, school_id, ai_used, page_num, respondent_key,
    18 AS question_num,
    'q18' AS question_code,
    cast(NULL as double) AS score,
    q18 AS text_answer
  FROM y18_base
)

SELECT
  *,
  CASE
    WHEN question_num BETWEEN 1 AND 12 THEN 'Q1-12 Experience'
    WHEN question_num BETWEEN 13 AND 17 THEN 'Q13-17 Tool Feedback'
    WHEN question_num = 18 THEN 'Q18 Free Text'
    ELSE 'Other'
  END AS question_group
FROM (
  SELECT * FROM n_long
  UNION ALL
  SELECT * FROM y12_long
  UNION ALL
  SELECT * FROM y13_17_long
  UNION ALL
  SELECT * FROM q18_long
);

--VALIDATION

SELECT * FROM fact_questionnaire_long;

SELECT
  school_id,
  ai_used,
  COUNT(DISTINCT respondent_key) AS respondent_count
FROM fact_questionnaire_long
GROUP BY school_id, ai_used
ORDER BY school_id, ai_used;


SELECT
  file_name,
  drive_file_id
FROM n_q1_q12_clean
WHERE school_id IS NULL
GROUP BY file_name, drive_file_id;

SELECT
  file_name,
  drive_file_id
FROM y_q1_q12_clean
WHERE school_id IS NULL
GROUP BY file_name, drive_file_id;

SELECT
  file_name,
  drive_file_id
FROM y_q13_q18_clean
WHERE school_id IS NULL
GROUP BY file_name, drive_file_id;

DELETE FROM y_q13_q18_raw
WHERE file_name="0.693";



